# Programación declarativa @ URJC
# Programación funcional
## Examen Convocatoria Ordinaria
## Curso 19-20

# Definiciones auxiliares

In [ ]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

### Algunas funciones sobre listas

In [ ]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](directSol: B)(composeSol: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
        
        // Reverse a list
        // e.g. List(1,2,3).reverse==List(3,2,1)
        def reverse: List[A]
        
        // Take the first `n` elements of the list
        // e.g. List(1,2,3).take(2) == List(1,2)
        //      List(1,2,3).take(0) == List()
        //      List(1,2,3).take(5) == List(1,2,3)
        def take(n: Int): List[A]
        
        // Drop the first `n` elements of the list 
        // e.g. List(1,2,3).drop(2) == List(3)
        //      List(1,2,3).drop(0) == List(1,2,3)
        //      List(1,2,3).drop(4) == List()
        def drop(n: Int): List[A]

        // List concatenation
        // e.g. List(1,2,3).concat(List(4,5)) == List(1,2,3,4,5)
        def concat(l: List[A]): List[A]
    }
}

### Definiciones sobre árboles binarios

In [ ]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

In [ ]:
object Tree{
    
    def void[A]: Tree[A] = 
        Empty()
    
    def leaf[A](a: A): Node[A] = 
        Node(Empty(), a, Empty())
    
    def right[A](a: A, tree: Tree[A]): Node[A] = 
        Node(Empty(), a, tree)
    
    def left[A](tree: Tree[A], a: A): Node[A] = 
        Node(tree, a, Empty())
    
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = 
        Node(left, a, right)
    
    def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
        tree match {
            case Empty() => 
                empty
            case Node(left, a, right) =>
                node(foldTree(left)(empty)(node),
                    a,
                    foldTree(right)(empty)(node))
        }
}

import Tree._

### Modelo de datos de películas

In [ ]:
case class MovieDatabase(
    films: Map[Film.Id, Film],
    users: Map[User.Id, User],
    ratings: Map[(Film.Id, User.Id), Rating])
    
case class Film(
    id: Film.Id, 
    title: String, 
    director: String,
    genre: String,
    year: Int,
    country: String)

object Film{
    type Id = Int
}

case class User(
    id: User.Id,
    name: String,
    registered: Int)
        
object User{
    type Id = Int
}
        
case class Rating(
    film: Film.Id,
    user: User.Id,
    score: Int)

In [ ]:
val moviedb: MovieDatabase = MovieDatabase(
    films = Map(
        1 -> Film(1, "Blade Runner", "Ridley Scott", "Sci-Fi", 1982, "United States"),
        2 -> Film(2, "Amanece, que no es poco", "José Luis Cuerda", "Comedy", 1989, "Spain"),
        3 -> Film(3, "El milagro de P. Tinto", "Javier Fesser", "Comedy", 1998, "Spain"),
        4 -> Film(4, "Mars Attacks!", "Tim Burton", "Sci-Fi", 1996, "United States"),
        5 -> Film(5, "2001: A Space Odyssey", "Stanley Kubrick", "Sci-Fi", 1968, "United Kingdom"),
        6 -> Film(6, "El crack Cero", "José Luis Garci", "Film noir", 2019, "Spain"),
        7 -> Film(7, "El crack", "José Luis Garci", "Film noir", 1981, "Spain"),
        8 -> Film(8, "The Maltese Falcon", "John Huston", "Film noir", 1941, "United States"),
        9 -> Film(9, "Chinatown", "Roman Polanski", "Film noir", 1974, "United States"),
        10 -> Film(10, "Batman v. Superman: Dawn of Justice", "Zack Snyder", "Sci-Fi", 2016, "United States"),
        11 -> Film(11, "Dumb and Dumber", "Peter Farrelly", "Comedy", 1994, "United States")
    ),
    users = Map(
        1 -> User(1, "Juan", 2000),
        2 -> User(2, "Alf", 1998),
        3 -> User(3, "Lola", 2004),
        4 -> User(4, "Lolo", 2018),
        5 -> User(5, "Dinu", 2005)),
    ratings = Map(
        (1,1) -> Rating(1,1,5),
        (1,2) -> Rating(1,2,1),
        (1,3) -> Rating(1,3,4),
        (1,4) -> Rating(1,4,3),
        (2,1) -> Rating(2,1,1),
        (2,4) -> Rating(2,4,1),
        (4,1) -> Rating(4,1,3),
        (5,4) -> Rating(5,4,2),
        (6,1) -> Rating(6,1,2),
        (7,1) -> Rating(7,1,3),
        (7,2) -> Rating(7,2,3),
        (7,3) -> Rating(7,3,3),
        (8,2) -> Rating(8,2,2),
        (9,1) -> Rating(9,1,1),
        (10,1) -> Rating(10,1,0),
        (10,3) -> Rating(10,3,0),
        (11,1) -> Rating(11,1,0),
        (11,2) -> Rating(11,2,1),
        (11,4) -> Rating(11,4,2)))

In [ ]:
object BasicQueries{
    
    // Entities
    
    def films(mdb: MovieDatabase): List[Film] =
        mdb.films.values.toList
    
    def filmIds(mdb: MovieDatabase): List[Film.Id] =
        mdb.films.keys.toList

    def getFilm(id: Film.Id)(mdb: MovieDatabase): List[Film] = 
        mdb.films.get(id).toList
    
    def userIds(mdb: MovieDatabase): List[User.Id] = 
        mdb.users.keys.toList
    
    def getUser(id: User.Id)(mdb: MovieDatabase): List[User] = 
        mdb.users.get(id).toList
    
    // 1-N relationships
    
    def films(dir: String)(mdb: MovieDatabase): List[Film.Id] = 
        mdb.films.filter(_._2.director == dir).map(_._1).toList
    
    // N-M relationships
    
    def ratings(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.values.toList
    
    def userRatings(user: User.Id)(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.filter(_._1._2 == user).values.toList
    
    def filmRatings(film: Film.Id)(mdb: MovieDatabase): List[Rating] = 
        mdb.ratings.filter(_._1._1 == film).values.toList
}

import BasicQueries._

# Ejercicio 1
__(0,5 puntos)__

Implementa la función `cnf` especificada por la siguiente signatura:

In [ ]:
def cnf[P, Q, R, S](a: Either[P, Q], b: Either[R, S]): 
        Either[(P, R), Either[(P, S), Either[(Q, R), (Q, S)]]] = 
    (a, b) match {
        case (Left(p), Left(r)) =>
            Left((p,r))
        case (Left(p), Right(s)) =>
            Right(Left((p,s)))
        case (Right(q), Left(r)) =>
            Right(Right(Left((q,r))))
        case (Right(q), Right(s)) =>
            Right(Right(Right((q,s))))
    }

# Ejercicio 2 
__(3,5 puntos)__

Un árbol binario de búsqueda es un árbol binario vacío o uno no vacío que cumple las siguientes condiciones:
* Los elementos del hijo izquierdo, caso de existir, son menores estrictamente que la raíz.
* Los elementos del hijo derecho, caso de existir, son mayores estrictamente que la raíz. 
* Sus hijos izquierdo y derecho son árboles binarios de búsqueda. 

Obsérvese que, según esta definición, en un árbol binario de búsqueda no hay elementos repetidos.

#### a) (1 punto)
Implementa la función `add` que añade un elemento a un árbol binario de búsqueda, de tal manera que el árbol resultante sea un árbol de búsqueda y contenga a dicho elemento. Por ejemplo:

In [ ]:
class TestAdd(
    add: (Tree[Int], Int) => Tree[Int]
) extends FlatSpec with Matchers{
    
    "add" should "work" in {
        add(void, 1) shouldBe leaf(1)
        add(leaf(1), 2) shouldBe right(1, leaf(2))
        add(leaf(1), 1) shouldBe leaf(1)
        add(left(leaf(2), 3), 1) shouldBe left(left(leaf(1), 2), 3)
        add(node(leaf(3), 4, leaf(6)), 5) shouldBe 
            node(leaf(3), 4, left(leaf(5), 6))
    }
}

In [ ]:
def add(tree: Tree[Int], i: Int): Tree[Int] = 
    tree match {
        case Empty() => leaf(i)
        case Node(left, root, right) => 
            if (i == root) tree
            else if (i < root) Node(add(left, i), root, right)
            else Node(left, root, add(right, i))
    }

In [ ]:
run(new TestAdd(add))

#### b) (1,5 puntos)

Implementa una función `toTree` que construya un árbol binario de búsqueda a partir de una lista de números. La implementación deberá realizarse con __recursión final__ (o recursión por cola) y utilizando la función `add` del apartado anterior. Además, se deberán satisfacer los siguientes casos de prueba:

In [ ]:
class TestToTree(
    toTree: List[Int] => Tree[Int]
) extends FlatSpec with Matchers{
    
    "toTree" should "work" in {
        toTree(List()) shouldBe void
        toTree(List(1)) shouldBe leaf(1)
        toTree(List(3,2,2)) shouldBe left(leaf(2), 3)
        toTree(List(2,3)) shouldBe right(2, leaf(3))
        toTree(List(1,2,4,3,2,5,1)) shouldBe 
            right(1, right(2, node(leaf(3), 4, leaf(5))))
    }
}

In [ ]:
def toTree(list: List[Int]): Tree[Int] = {
    def auxToTree(out: Tree[Int], aux: List[Int]): Tree[Int] = 
        aux match {
            case Nil => out
            case head :: tail => 
                auxToTree(add(out, head), tail)
        }
    auxToTree(void, list)
}

In [ ]:
run(new TestToTree(toTree))

#### c) (1 punto)
Implementa la función `toTree` utilizando __`foldLeft`__.

In [ ]:
def toTree(list: List[Int]): Tree[Int] = 
    list.foldLeft(void[Int])(add)

In [ ]:
run(new TestToTree(toTree))

# Ejercicio 3
__(2,5 puntos)__

Se desea implementar una función `compress` que reemplace las subsecuencias de elementos repetidos en una lista de entrada por pares formados por los elementos y las longitudes de dichas subsecuencias. Por ejemplo:

In [ ]:
class TestCompress(
    compress: List[Char] => List[(Char, Int)]
) extends FlatSpec with Matchers{
    
    "compress" should "work" in {
        compress(List()) shouldBe 
            List()
        compress(List('a','a','a','c','c','b','z','z')) shouldBe 
            List(('a',3),('c',2),('b',1),('z',2))
        compress(List('a','b','c')) shouldBe 
            List(('a',1),('b',1),('c',1))
        compress(List('a','a','a','c','c','a', 'b','z','z')) shouldBe 
            List(('a',3),('c',2),('a', 1), ('b',1),('z',2))
        compress(List('k','k','a','ñ','ñ','ñ','h', 'k')) shouldBe 
            List(('k',2),('a',1),('ñ',3),('h',1),('k', 1))
    }
}

__a) (1,5 puntos)__ Implementa la función `compress` de manera recursiva (__sin__ recursividad por cola).

In [ ]:
def compress[A](list: List[A]): List[(A, Int)] = 
    list match {
        case Nil => 
            List()
        case head :: tail => 
            compress(tail) match {
                case (`head`, n) :: tailT => 
                    (head, n+1) :: tailT
                case tailSol => 
                    (head, 1) :: tailSol
            }
    }

In [ ]:
run(new TestCompress(compress))

__b) (1 punto)__ Implementar la función `compress` utilizando __`foldRight`__.

In [ ]:
def compress[A](list: List[A]): List[(A, Int)] = 
    list.foldRight(List[(A, Int)]()){
        case (head, (headT, n) :: tailT) if headT == head =>
            (headT, n+1) :: tailT
        case (head, tailSol) => 
            (head, 1) :: tailSol
    }

In [ ]:
run(new TestCompress(compress))

# Ejercicio 4
__(1 punto/modelos A, C)__ 

__(2 puntos/modelos B, D)__

Dada una lista, un elemento y un número positivo, la función `insertAt` inserta el elemento en la posición especificada de la lista, o en la última posición, en caso de que la lista resultante no tenga el suficiente número de elementos. Considérese que la cabeza de la lista se encuentra en la posición uno. Por ejemplo:

In [ ]:
class TestInsertAt(
    insertAt: (Char, Int, List[Char]) => List[Char]
) extends FlatSpec with Matchers{
    "insertAt" should "work" in {
        insertAt('a', 3, List('a','b','c','d','e')) shouldBe 
            List('a','b','a', 'c','d','e')
        insertAt('a', 4, List('a','b')) shouldBe 
            List('a','b','a')
        insertAt('a', 3, List('a','b')) shouldBe 
            List('a','b','a')
        insertAt('a', 1, List('d','b','e')) shouldBe 
            List('a','d','b','e')
    }
}

__a) (1 punto)__ Se pide utilizar las funciones de la librería de colecciones de Scala `take` y `drop` para implementar la función `insertAt`. 

In [ ]:
def insertAt[A](a: A, n: Int, list: List[A]): List[A] = 
    list.take(n-1) ++ (a :: list.drop(n-1))

In [ ]:
run(new TestInsertAt(insertAt))

__b) (1 punto / modelos B, D)__ Implementa la función mediante `foldLeft` y `reverse`, __sin__ utilizar ninguna otra función de la librería de colecciones de Scala.

In [ ]:
def insertAt[A](a: A, n: Int, list: List[A]): List[A] = {
    val (out, length) = list.foldLeft((List[A](), 0)){
        case ((out, l), e) if l+1 == n => 
            (e :: a :: out, l+1)
        case ((out, l), e) => 
            (e :: out, l+1)
    }
    (if (length < n) a :: out else out).reverse
}

In [ ]:
run(new TestInsertAt(insertAt))

# Ejercicio 5

__(2,5 puntos/ modelos A, B)__

__(1,5 puntos/ modelos B, D)__

Considere la siguiente implementación imperativa de una función sobre el modelo de datos de películas:

In [ ]:
def mistery(id: Film.Id, year: Int)(
        mdb: MovieDatabase): List[(String, Int)] = {
    var out: List[(String, Int)] = List()
    for (rating <- filmRatings(id)(mdb))
        for (user <- getUser(rating.user)(mdb))
            if (user.registered >= year)
                out = (user.name, user.registered) :: out
    out.reverse
}

__a) (0,2 puntos)__ Describa en lenguaje natural, brevemente y de manera precisa, la especificación de dicha función.

_Dados una película y una fecha, la función devuelve las fechas de registro y nombres de los usuarios que han valorado dicha película registrados con posterioridad a la fecha de entrada._

#### b) (1,3 puntos)
Implementa la función `mistery` utilizando las funciones de orden superior `flatMap`, `filter` y `map`.

In [ ]:
class TestMistery(
    misteryHOF: (Film.Id, Int) => MovieDatabase => List[(String, Int)]
) extends FlatSpec with Matchers {
    
    "misteryHOF" should "work" in {
        misteryHOF(1, 2001)(moviedb) shouldBe 
            mistery(1, 2001)(moviedb)
        
        misteryHOF(7, 1999)(moviedb) shouldBe 
            mistery(7, 1999)(moviedb)
    }
}

In [ ]:
def misteryHOF(id: Film.Id, year: Int)(mdb: MovieDatabase): List[(String, Int)] = 
    filmRatings(id)(mdb)
        .flatMap(rating => getUser(rating.user)(mdb))
        .filter(_.registered >= year)
        .map(user => (user.name, user.registered))

In [ ]:
run(new TestMistery(mistery))

#### c) (1 punto / modelos A, B)
Implementa la función `mistery` utilizando funciones de orden superior, pero __sin__ utilizar la función `filter`.

In [ ]:
def misteryHOF(id: User.Id, year: Int)(mdb: MovieDatabase): List[(String, Int)] = 
    filmRatings(id)(mdb)
        .flatMap(rating => getUser(rating.user)(mdb))
        .flatMap(user => 
            if (user.registered >= year) 
                List((user.name, user.registered))
            else List())

In [ ]:
run(new TestMistery(misteryHOF))